# introduction to query execution plans

whenever we write a sql query, the database system first parses our code to generate a query tree.

then the optimizer modifies the query tree either based on relational algebra or cost-based optimization to generate an execution plan.

the execution plan is a sequence of operations that the database system will perform to execute the query.

the operations are things like "scan this table", "filter out rows that don't satisfy this condition", "join these two tables", and so on.

using the `explain` command, we can see the execution plan that the system came up with for our query.

In [41]:
!psql postgres -c "\h explain"

Command:     EXPLAIN
Description: show the execution plan of a statement
Syntax:
EXPLAIN [ ( option [, ...] ) ] statement
EXPLAIN [ ANALYZE ] [ VERBOSE ] statement

where option can be one of:

    ANALYZE [ boolean ]
    VERBOSE [ boolean ]
    COSTS [ boolean ]
    SETTINGS [ boolean ]
    GENERIC_PLAN [ boolean ]
    BUFFERS [ boolean ]
    WAL [ boolean ]
    TIMING [ boolean ]
    SUMMARY [ boolean ]
    FORMAT { TEXT | XML | JSON | YAML }

URL: https://www.postgresql.org/docs/16/sql-explain.html



since the execution plan is a tree, you have to read it from the leaf nodes to the root node.

each child node provides the input to the parent node.

the root node is the final result of the query that aggregates all the intermediate results (and costs).

learn more through the following resources:

_docs_

- https://www.postgresql.org/docs/current/sql-explain.html
- https://www.postgresql.org/docs/9.5/using-explain.html
- overview: https://www.postgresql.org/docs/current/index.html
- performance tips: https://www.postgresql.org/docs/current/performance-tips.html
- glossary: https://www.pgmustard.com/docs/explain

_visualization tools_

- https://explain.dalibo.com/
     - source: https://github.com/dalibo/pev2
- https://www.pgexplain.dev/ (pev2 but with a backend)
     - source: https://github.com/lfborjas/postgres-explain-visualizer
- https://tatiyants.com/pev/
     - source: https://github.com/AlexTatiyants/pev/
- https://explain.depesz.com/
- https://explain-postgresql.com/ (not so good)

# 1) what is the default triangle-join plan?

what is the most common execution plan for the following query: $r \bowtie s \bowtie t$?

In [46]:
# reset the environment
!chmod +x ./reset.sh && ./reset.sh >> /dev/null

./reset.sh: line 10: return: can only `return' from a function or sourced script
Did not find any relations.
ERROR:  cannot drop the currently open database
ERROR:  current user cannot be dropped


In [43]:
# print all tables
!psql postgres -c "\dt+"

                                      List of relations
 Schema |    Name     | Type  |  Owner  | Persistence | Access method |  Size   | Description 
--------+-------------+-------+---------+-------------+---------------+---------+-------------
 public | badges      | table | sueszli | permanent   | heap          | 2144 kB | 
 public | comments    | table | sueszli | permanent   | heap          | 8288 kB | 
 public | posthistory | table | sueszli | permanent   | heap          | 31 MB   | 
 public | postlinks   | table | sueszli | permanent   | heap          | 152 kB  | 
 public | posts       | table | sueszli | permanent   | heap          | 15 MB   | 
 public | r           | table | sueszli | permanent   | heap          | 1024 kB | 
 public | s           | table | sueszli | permanent   | heap          | 128 kB  | 
 public | t           | table | sueszli | permanent   | heap          | 1024 kB | 
 public | tags        | table | sueszli | permanent   | heap          | 56 kB   | 
 public

In [44]:
# print schema of r, s, t tables
!psql postgres -c "\d r"
!psql postgres -c "\d s"
!psql postgres -c "\d t"

                 Table "public.r"
 Column |  Type   | Collation | Nullable | Default 
--------+---------+-----------+----------+---------
 a      | integer |           |          | 
 b      | integer |           |          | 

                 Table "public.s"
 Column |  Type   | Collation | Nullable | Default 
--------+---------+-----------+----------+---------
 b      | integer |           |          | 
 c      | integer |           |          | 

                 Table "public.t"
 Column |  Type   | Collation | Nullable | Default 
--------+---------+-----------+----------+---------
 a      | integer |           |          | 
 c      | integer |           |          | 



In [101]:
!psql postgres -c "explain analyze SELECT a,b,c FROM r NATURAL JOIN s NATURAL JOIN t;"

                                                           QUERY PLAN                                                           
--------------------------------------------------------------------------------------------------------------------------------
 Merge Join  (cost=52724.37..89027.34 rows=2010918 width=12) (actual time=127.162..322.327 rows=2008672 loops=1)
   Merge Cond: ((t.c = s.c) AND (t.a = r.a))
   ->  Sort  (cost=1717.77..1767.77 rows=20000 width=8) (actual time=7.572..8.591 rows=10264 loops=1)
         Sort Key: t.c, t.a
         Sort Method: quicksort  Memory: 1394kB
         ->  Seq Scan on t  (cost=0.00..289.00 rows=20000 width=8) (actual time=0.009..1.607 rows=20000 loops=1)
   ->  Materialize  (cost=51006.60..53076.46 rows=413972 width=12) (actual time=119.581..199.888 rows=2216058 loops=1)
         ->  Sort  (cost=51006.60..52041.53 rows=413972 width=12) (actual time=119.578..142.644 rows=413972 loops=1)
               Sort Key: s.c, r.a
               Sort Met

In [103]:
!psql postgres -c "explain (costs off) SELECT a,b,c FROM r NATURAL JOIN s NATURAL JOIN t;"

                 QUERY PLAN                  
---------------------------------------------
 Merge Join
   Merge Cond: ((t.c = s.c) AND (t.a = r.a))
   ->  Sort
         Sort Key: t.c, t.a
         ->  Seq Scan on t
   ->  Materialize
         ->  Sort
               Sort Key: s.c, r.a
               ->  Hash Join
                     Hash Cond: (r.b = s.b)
                     ->  Seq Scan on r
                     ->  Hash
                           ->  Seq Scan on s
(13 rows)



the default join strategy for $r \bowtie s \bowtie t$:

1. hash join $r \bowtie s$.

    with $s$ in memory.
    
    then sort the result and materialize it (turn it into a table so it doesn't have to be recomputed).

2. sort $t$ and merge join the result of step 1.


<br>

_what is a hash join?_

assume we want to equi join: $R \bowtie_{\text{A}=\text{B}} S$

- 1) partition phase:
	- find a hash function that can map values in the join columns to a buffer frame index between $[1;B\text{-}1]$. → the buffer frames we map the rows to are called "buckets" and the 1 remaining buffer frame is used to read new pages in.
	- read each page $p_R$ of $R$ to memory. then hash the join value of each row to find the right bucket to store a pointer in. → if buffer frames overflow, write them back to disk.
	- repeat for $p_S$ of $S$.
	- total cost: $2 \cdot (b_R  + b_S)$ → factor of 2 because of initial reading and writing back the potentially full buckets to disk.
- 2) probing phase:
	- assuming $R_i$ and $S_i$ are all rows in the $i$ th-bucket (and $R_i$ is the smaller one of them): read $R_i$ to $B\text-2$ buffer frames. → if not possible, either hash recursively or try another algorithm. the 2 remaining buffer frames are used to read new $S_i$ pages in and store the final result.
	- read each page of $S_i$ into memory. then check each row for matches with $R_i$.
	- if a matching row is found, write it into the buffer frame dedicated to results.
	- total cost: $b_i  + b_s$
- **total cost of both phases**: $3 \cdot (b_R  + b_S)$